In [1]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import scipy.ndimage
import cv2
import os
from matplotlib import pyplot as plt
import sys

# Get the home directory
home_directory = os.path.expanduser('~')

# Append the parent directory to the Python path
sys.path.append(os.path.join(home_directory, 'Ashish/UCAN', 'UCAN-PET-CT-image-data-handling-pipeline'))

from Utils import utils

def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

In [2]:
def get_proj_after_mask(img):
    
    '''

    Function to get 3D masks for CT scans to segment out the exact tissue type.

    '''

    pix_array=sitk.GetArrayFromImage(img)
    max_i, min_i=float(pix_array.max()),float(pix_array.min())

    #multiply= sitk.MultiplyImageFilter()
    #if hu_type == 'Bone' or hu_type == 'bone' or hu_type == 'B':
    bone_mask = sitk.BinaryThreshold(
    img, lowerThreshold=200, upperThreshold=max_i,insideValue=1, outsideValue=0
    )
    
    tracer_contrast_mask = sitk.BinaryThreshold(
    img, lowerThreshold=151, upperThreshold=199,insideValue=1, outsideValue=0
    )

    #bone_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        # path= img_n + r'_{0}_image.nii'.format(modality + '_' + type)
        # save_as_gz(op_img,path)
    #elif hu_type == 'Lean Tissue' or hu_type == 'lean' or hu_type == 'LT':
    lean_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-29, upperThreshold=150, insideValue=1, outsideValue=0
    )
    #lean_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))

    #elif hu_type == 'Adipose' or hu_type == 'adipose' or hu_type == 'AT':
    adipose_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-199, upperThreshold=-30, insideValue=1, outsideValue=0
    )
    #adipose_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        
    #elif hu_type == 'Air' or hu_type == 'A':
    air_mask = sitk.BinaryThreshold(
    img, lowerThreshold=min_i, upperThreshold=-191, insideValue=1, outsideValue=0
    )
    #air_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
    
    return bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask

In [3]:
CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/npr100169878450_SUV_CT/20130412/CT.nii.gz"

In [4]:
CT_img = sitk.ReadImage(CT_path)

In [5]:
bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

In [6]:
lean_mask_arr = sitk.GetArrayFromImage(lean_mask)
np.unique(lean_mask_arr)

array([0, 1], dtype=uint8)

In [7]:
lean_mask_arr.sum()

2479840

In [8]:
np.unique(lean_mask_arr, return_counts=True)

(array([0, 1], dtype=uint8), array([16328992,  2479840]))

In [9]:
voxel_spacing = [2.0364201068878174, 2.0364201068878174, 3.0]

In [10]:
# calculate lean density of lean muscle # multiply with 1e-3 to covert mm3 to cm3 (ml)  # ml * 1e-3 litres
density = lean_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6
density

30.851740413634367

In [11]:
# calculate bone skeleton density
bone_mask_arr = sitk.GetArrayFromImage(bone_mask)
print(np.unique(bone_mask_arr))
print(bone_mask_arr.sum())
print(np.unique(bone_mask_arr, return_counts=True))
bone_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6

[0 1]
266094
(array([0, 1], dtype=uint8), array([18542738,   266094]))


3.3104809236183073

In [12]:
multiply= sitk.MultiplyImageFilter()
CT_tracer_contrast = multiply.Execute(CT_img,sitk.Cast(tracer_contrast_mask,CT_img.GetPixelID()))

CT_ptype = 'mean'
SUV_ptype = 'max'
angle = 90

raw_projections_path = "/home/ashish/Ashish/UCAN/Testing/test_new_projections/"

utils.get_2D_projections(CT_tracer_contrast, 'CT', CT_ptype, angle, invert_intensity= True, img_n=raw_projections_path)
    

Finished generating 3 - mean intensity 2D projections from the CT volume image! 


In [30]:
resampled_directory_list = []
resampled_SUV_CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/"

for dirs, subdirs, files in os.walk(resampled_SUV_CT_path):
    for file in files:
        file_path = str(os.path.join(dirs, file))
        file_path = file_path.replace('\\','/')
        resampled_directory_list.append(file_path)

resampled_directory_df = pd.DataFrame(resampled_directory_list, columns=['directory'])
resampled_directory_df[['source_directory', 'patient_directory', 'scan_date', 'SUV_CT']] = resampled_directory_df['directory'].str.rsplit(pat='/', n=3, expand=True)
resampled_directory_df[['npr', 'extra']] = resampled_directory_df['patient_directory'].str.split(pat='_', n=1, expand=True)
resampled_directory_df.drop(columns=['directory','extra', 'SUV_CT'], inplace=True)
resampled_directory_df.drop_duplicates(inplace=True)

utils.display_full(resampled_directory_df.head(2))

count = 0
# Generate the raw 2D projections
for index, row in resampled_directory_df.iterrows():
    raw_projections_path = "/home/ashish/Ashish/UCAN/Results/test_new_proj/"
    angle = 90

    CTnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'CT.nii.gz'
    print("CTnii_path", CTnii_path)
    
    CT_img =sitk.ReadImage(CTnii_path)
    sitk.WriteImage(CT_img, raw_projections_path + '/test_CT.nii.gz')

    SUVnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'SUV.nii.gz'
    SUV_img =sitk.ReadImage(SUVnii_path)
    print("SUVnii_path", SUVnii_path)
    sitk.WriteImage(SUV_img, raw_projections_path + '/test_SUV.nii.gz')
    break
    bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

    multiply= sitk.MultiplyImageFilter()
    #try:

    CT_tracer_contrast = multiply.Execute(CT_img,sitk.Cast(tracer_contrast_mask,CT_img.GetPixelID()))
 
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MaxIP_')
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MeanIP_')
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MinIP_')

    SUV_tracer_contrast = multiply.Execute(SUV_img,sitk.Cast(tracer_contrast_mask,SUV_img.GetPixelID()))
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MaxIP_')
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MeanIP_')
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MinIP_')


    CT_bone = multiply.Execute(CT_img,sitk.Cast(bone_mask,CT_img.GetPixelID()))
    utils.get_2D_projections(CT_bone, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MaxIP_')
    utils.get_2D_projections(CT_bone, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MinIP_')

    CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask,CT_img.GetPixelID()))
    utils.get_2D_projections(CT_lean, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MaxIP_')
    utils.get_2D_projections(CT_lean, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MinIP_')
    
    count += 1
    print(str(row['npr']) + '_' + str(row['scan_date']))
    print("count: ", count)
    #except: 
        #print(str(row['npr']) + '_' + str(row['scan_date']))
    #    pass

    if count > 10:
        break

                            source_directory       patient_directory scan_date              npr
0  /home/ashish/Ashish/UCAN/Resampled_SUV_CT  npr865130139443_SUV_CT  20210622  npr865130139443
2  /home/ashish/Ashish/UCAN/Resampled_SUV_CT  npr865130139443_SUV_CT  20211126  npr865130139443
CTnii_path /home/ashish/Ashish/UCAN/Resampled_SUV_CT/npr865130139443_SUV_CT/20210622/CT.nii.gz


** ERROR (nifti_image_write_engine): cannot open output file '/home/ashish/Ashish/UCAN/Results/test_new_proj//test_CT.nii.gz'


RuntimeError: Exception thrown in SimpleITK ImageFileWriter_Execute: /tmp/SimpleITK-build/ITK/Modules/IO/NIFTI/src/itkNiftiImageIO.cxx:2254:
ITK ERROR: NiftiImageIO(0x4445710): ERROR: nifti library failed to write image/home/ashish/Ashish/UCAN/Results/test_new_proj//test_CT.nii.gz

In [18]:
resampled_directory_df.head()

,source_directory,patient_directory,scan_date,npr
0,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr865130139443_SUV_CT,20210622,npr865130139443
2,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr865130139443_SUV_CT,20211126,npr865130139443
4,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr382894159583_SUV_CT,20150127,npr382894159583
6,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr382894159583_SUV_CT,20140828,npr382894159583
8,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr717322658733_SUV_CT,20180718,npr717322658733
